# Snowflake for Data Scientists: TPC-H 데이터 분석 실습

이 노트북은 Snowflake를 처음 사용하는 Data Scientist를 위한 실습 가이드입니다.

## 학습 목표
1. **Snowpark Python**을 활용한 데이터 탐색
2. **시각화**: 매출 트렌드, 지역별 분석, 배송 패턴
3. **머신러닝**: Snowflake ML을 활용한 고객 세그먼트 예측

## 데이터셋: TPC-H
TPC-H는 의사결정 지원 시스템 벤치마크로, 실제 비즈니스 시나리오를 모방한 8개의 테이블로 구성됩니다:
- `CUSTOMER`: 고객 정보 (150,000건)
- `ORDERS`: 주문 정보 (1,500,000건)
- `LINEITEM`: 주문 상세 (6,001,215건)
- `PART`: 제품 정보
- `SUPPLIER`: 공급업체
- `NATION` / `REGION`: 지역 정보

---
## 1. 환경 설정 및 Snowpark 세션 연결

### 1.1 패키지 설치 (필수!)

이 노트북을 실행하기 전에 **필요한 패키지를 먼저 추가**해야 합니다.

#### 설치 방법:
1. 화면 우측 상단의 **Packages** 버튼 클릭
2. 검색창에서 아래 패키지를 각각 검색하여 추가:
   - `matplotlib`
   - `scikit-learn`
   - `altair`
   - `snowflake-ml-python`
3. 패키지가 추가되면 자동으로 Notebook 환경에 설치됩니다

![Packages Button](https://docs.snowflake.com/en/_images/notebooks-packages-button.png)

> **Note**: `pandas`, `numpy`, `snowflake-snowpark-python`은 기본 제공되므로 별도 설치 불필요합니다.

### 1.2 라이브러리 Import
Snowflake Notebook에서는 `get_active_session()`으로 현재 세션에 바로 연결됩니다.
로컬 환경에서는 `Session.builder.configs(...).create()`를 사용합니다.

In [ ]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

try:
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
    print("Using Snowsight session")
except:
    import snowflake.connector
    conn = snowflake.connector.connect(connection_name=os.getenv("SNOWFLAKE_CONNECTION_NAME") or "demo33")
    session = Session.builder.configs({"connection": conn}).create()
    print("Using local connection")
    session.sql("USE DATABASE DEMO_DB").collect()
    session.sql("USE SCHEMA PUBLIC").collect()

print(f"Connected to: {session.get_current_database()}.{session.get_current_schema()}")

---
## 2. 데이터 탐색 (EDA)

### 2.1 테이블 개요
Snowpark DataFrame은 **지연 실행(Lazy Evaluation)** 됩니다. 
`.collect()` 또는 `.to_pandas()`를 호출하기 전까지 실제 쿼리가 실행되지 않습니다.

> **핵심 개념**: Snowpark는 Python 코드를 SQL로 변환하여 Snowflake 엔진에서 실행합니다.
> 데이터가 클라이언트로 이동하지 않아 대용량 데이터 처리에 효율적입니다.

In [ ]:
orders_df = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS")
lineitem_df = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.LINEITEM")
customer_df = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER")
nation_df = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION")
region_df = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.REGION")

print("=== 테이블 행 수 ===")
print(f"ORDERS: {orders_df.count():,} rows")
print(f"LINEITEM: {lineitem_df.count():,} rows")
print(f"CUSTOMER: {customer_df.count():,} rows")

### 2.2 스키마 확인
Snowpark DataFrame의 `.schema`로 컬럼 정보를 확인할 수 있습니다.

In [ ]:
print("=== ORDERS 테이블 스키마 ===")
for field in orders_df.schema.fields:
    print(f"  {field.name}: {field.datatype}")

### 2.3 샘플 데이터 미리보기
`.limit()`과 `.to_pandas()`를 조합하여 샘플 데이터를 확인합니다.

In [ ]:
orders_df.limit(5).to_pandas()

### 2.4 기본 통계량
`.describe()`로 수치형 컬럼의 통계량을 확인합니다.

In [ ]:
orders_df.describe().to_pandas()

---
## 3. 데이터 시각화

### 3.1 월별 매출 트렌드

Snowpark에서 집계 후 Pandas로 변환하여 시각화합니다.

> **Best Practice**: 집계는 Snowflake에서, 시각화는 클라이언트에서 처리하면 효율적입니다.

In [ ]:
monthly_revenue = (
    orders_df
    .with_column("YEAR_MONTH", F.date_trunc("MONTH", F.col("O_ORDERDATE")))
    .group_by("YEAR_MONTH")
    .agg(F.sum("O_TOTALPRICE").alias("TOTAL_REVENUE"))
    .order_by("YEAR_MONTH")
    .to_pandas()
)

monthly_revenue.head()

In [ ]:
chart = alt.Chart(monthly_revenue).mark_line(point=True).encode(
    x=alt.X("YEAR_MONTH:T", title="Date"),
    y=alt.Y("TOTAL_REVENUE:Q", title="Revenue ($)", axis=alt.Axis(format="~s")),
    tooltip=["YEAR_MONTH:T", alt.Tooltip("TOTAL_REVENUE:Q", format="$,.0f")]
).properties(
    title="Monthly Revenue Trend",
    width=700,
    height=400
)
chart

### 3.2 지역별 매출 분포

여러 테이블을 JOIN하여 지역별 매출을 분석합니다.

> **Snowpark JOIN**: `.join()` 메서드로 DataFrame을 조인합니다. SQL JOIN과 동일하게 동작합니다.

In [ ]:
region_revenue = (
    orders_df
    .join(customer_df, orders_df["O_CUSTKEY"] == customer_df["C_CUSTKEY"])
    .join(nation_df, customer_df["C_NATIONKEY"] == nation_df["N_NATIONKEY"])
    .join(region_df, nation_df["N_REGIONKEY"] == region_df["R_REGIONKEY"])
    .group_by("R_NAME")
    .agg(
        F.sum("O_TOTALPRICE").alias("TOTAL_REVENUE"),
        F.count("O_ORDERKEY").alias("ORDER_COUNT")
    )
    .to_pandas()
)

region_revenue

In [ ]:
chart = alt.Chart(region_revenue).mark_bar().encode(
    x=alt.X("R_NAME:N", title="Region", sort="-y"),
    y=alt.Y("TOTAL_REVENUE:Q", title="Total Revenue ($)", axis=alt.Axis(format="~s")),
    color=alt.Color("R_NAME:N", legend=None),
    tooltip=["R_NAME", alt.Tooltip("TOTAL_REVENUE:Q", format="$,.0f"), "ORDER_COUNT"]
).properties(
    title="Revenue by Region",
    width=500,
    height=350
)
chart

### 3.3 배송 모드별 분석

LINEITEM 테이블에서 배송 모드(L_SHIPMODE)별 패턴을 분석합니다.

In [ ]:
shipmode_analysis = (
    lineitem_df
    .group_by("L_SHIPMODE")
    .agg(
        F.count("*").alias("COUNT"),
        F.avg("L_QUANTITY").alias("AVG_QUANTITY"),
        F.sum("L_EXTENDEDPRICE").alias("TOTAL_PRICE")
    )
    .order_by(F.col("TOTAL_PRICE").desc())
    .to_pandas()
)

shipmode_analysis

In [ ]:
chart = alt.Chart(shipmode_analysis).mark_bar().encode(
    x=alt.X("L_SHIPMODE:N", title="Ship Mode", sort="-y"),
    y=alt.Y("COUNT:Q", title="Number of Items"),
    color=alt.Color("L_SHIPMODE:N", legend=None),
    tooltip=["L_SHIPMODE", "COUNT", alt.Tooltip("AVG_QUANTITY:Q", format=".2f")]
).properties(
    title="Items by Ship Mode",
    width=500,
    height=350
)
chart

### 3.4 주문 우선순위별 평균 주문액

주문 우선순위(O_ORDERPRIORITY)에 따른 평균 주문 금액을 분석합니다.

In [ ]:
priority_analysis = (
    orders_df
    .group_by("O_ORDERPRIORITY")
    .agg(
        F.count("*").alias("ORDER_COUNT"),
        F.avg("O_TOTALPRICE").alias("AVG_ORDER_VALUE"),
        F.sum("O_TOTALPRICE").alias("TOTAL_REVENUE")
    )
    .order_by("O_ORDERPRIORITY")
    .to_pandas()
)

priority_analysis

In [ ]:
chart = alt.Chart(priority_analysis).mark_bar().encode(
    x=alt.X("O_ORDERPRIORITY:N", title="Order Priority"),
    y=alt.Y("AVG_ORDER_VALUE:Q", title="Avg Order Value ($)"),
    color=alt.Color("O_ORDERPRIORITY:N", legend=None),
    tooltip=["O_ORDERPRIORITY", alt.Tooltip("AVG_ORDER_VALUE:Q", format="$,.2f"), "ORDER_COUNT"]
).properties(
    title="Average Order Value by Priority",
    width=500,
    height=350
)
chart

---
## 4. 머신러닝: 고객 시장 세그먼트 예측

### 4.1 문제 정의

고객의 주문 패턴(주문 횟수, 총 지출액, 평균 주문액)을 기반으로 **시장 세그먼트(C_MKTSEGMENT)**를 예측하는 분류 모델을 구축합니다.

시장 세그먼트는 5개 카테고리로 구성:
- AUTOMOBILE, BUILDING, FURNITURE, HOUSEHOLD, MACHINERY

> **Snowflake ML**: Snowpark ML을 사용하면 모델 학습부터 배포까지 Snowflake 내에서 처리됩니다.
> 데이터가 Snowflake를 벗어나지 않아 보안과 거버넌스가 유지됩니다.

### 4.2 Feature Engineering

고객별 주문 통계를 집계하여 피처를 생성합니다.

In [ ]:
customer_features = (
    orders_df
    .join(customer_df, orders_df["O_CUSTKEY"] == customer_df["C_CUSTKEY"])
    .group_by("C_CUSTKEY", "C_MKTSEGMENT", "C_NATIONKEY", "C_ACCTBAL")
    .agg(
        F.count("O_ORDERKEY").alias("ORDER_COUNT"),
        F.sum("O_TOTALPRICE").alias("TOTAL_SPENT"),
        F.avg("O_TOTALPRICE").alias("AVG_ORDER_VALUE"),
        F.max("O_TOTALPRICE").alias("MAX_ORDER_VALUE"),
        F.min("O_TOTALPRICE").alias("MIN_ORDER_VALUE")
    )
)

print(f"Features 생성 완료: {customer_features.count():,} rows")
customer_features.limit(5).to_pandas()

### 4.3 데이터 분할 (Train/Test Split)

Snowpark ML의 `random_split()`을 사용하여 학습/테스트 세트로 분할합니다.

In [ ]:
train_df, test_df = customer_features.random_split([0.8, 0.2], seed=42)

print(f"Train set: {train_df.count():,} rows")
print(f"Test set: {test_df.count():,} rows")

### 4.4 모델 학습: Random Forest Classifier

Snowpark ML의 `RandomForestClassifier`를 사용합니다.

> **핵심 개념**: 모델 학습이 Snowflake의 컴퓨팅 리소스에서 실행됩니다.
> 대용량 데이터도 로컬 메모리 제약 없이 처리할 수 있습니다.

In [ ]:
from snowflake.ml.modeling.ensemble import RandomForestClassifier

feature_cols = ["ORDER_COUNT", "TOTAL_SPENT", "AVG_ORDER_VALUE", "MAX_ORDER_VALUE", "MIN_ORDER_VALUE", "C_ACCTBAL", "C_NATIONKEY"]
label_col = "C_MKTSEGMENT"

rf_model = RandomForestClassifier(
    input_cols=feature_cols,
    label_cols=[label_col],
    output_cols=["PREDICTED_SEGMENT"],
    n_estimators=100,
    max_depth=10,
    random_state=42
)

rf_model.fit(train_df)
print("Model training completed!")

### 4.5 모델 평가

In [ ]:
predictions = rf_model.predict(test_df)

# 예측 결과 컬럼 확인
print("Prediction columns:", predictions.columns)

# 실제 컬럼명 사용
prediction_col = "PREDICTED_SEGMENT" if "PREDICTED_SEGMENT" in predictions.columns else [col for col in predictions.columns if "PREDICT" in col.upper()][0]

predictions_pd = predictions.select("C_MKTSEGMENT", prediction_col).to_pandas()

accuracy = (predictions_pd["C_MKTSEGMENT"] == predictions_pd[prediction_col]).mean()
print(f"Accuracy: {accuracy:.2%}")

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

# 이전 셀에서 사용한 prediction_col 사용
prediction_col = "PREDICTED_SEGMENT" if "PREDICTED_SEGMENT" in predictions_pd.columns else [col for col in predictions_pd.columns if "PREDICT" in col.upper()][0]

cm = confusion_matrix(predictions_pd["C_MKTSEGMENT"], predictions_pd[prediction_col])
labels = sorted(predictions_pd["C_MKTSEGMENT"].unique())

cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print("Confusion Matrix:")
cm_df

### 4.6 Feature Importance

어떤 피처가 예측에 가장 중요한지 확인합니다.

In [ ]:
importance_df = pd.DataFrame({
    "feature": feature_cols,
    "importance": rf_model.to_sklearn().feature_importances_
}).sort_values("importance", ascending=False)

chart = alt.Chart(importance_df).mark_bar().encode(
    x=alt.X("importance:Q", title="Importance"),
    y=alt.Y("feature:N", sort="-x", title="Feature"),
    color=alt.Color("feature:N", legend=None)
).properties(
    title="Feature Importance",
    width=500,
    height=300
)
chart

---
## 5. 요약 및 다음 단계

### 이 노트북에서 학습한 내용:
1. **Snowpark Python**: SQL 없이 Python으로 Snowflake 데이터 처리
2. **지연 실행(Lazy Evaluation)**: 효율적인 대용량 데이터 처리
3. **시각화**: Snowflake 집계 + Altair/Matplotlib 시각화
4. **Snowpark ML**: Snowflake 내에서 ML 모델 학습 및 예측

### 추가 학습 권장:
- Snowflake Cortex LLM Functions
- Snowflake Feature Store
- Model Registry를 통한 모델 버전 관리
- Streamlit in Snowflake로 대시보드 배포